In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import re
from tqdm.notebook import tqdm
from time import sleep

In [2]:
URL = 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N'

res = requests.get(URL)

In [3]:
res.status_code

200

In [4]:
soup = BS(res.text)

In [5]:
print(soup.prettify())

<!DOCTYPE html>
<!--[if lte IE 8]><html class="no-js lte-ie9 lte-ie8" lang="en" dir="ltr"><![endif]-->
<!--[if IE 9]><html class="no-js lte-ie9" lang="en" dir="ltr"><![endif]-->
<!--[if gt IE 9]><!-->
<html class="no-js" dir="ltr" lang="en">
 <!--<![endif]-->
 <head>
  <!-- Google Tag Manager -->
  <script>
   (function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
      new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
      j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
      'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
      })(window,document,'script','dataLayer','GTM-MKPRXNV');
  </script>
  <!-- End Google Tag Manager -->
  <meta charset="utf-8"/>
  <script type="text/javascript">
   window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"NRJS-b7f93d6e9afef1935a5","applicationID":"1109265918","transactionName":"dFpeQkdcWQoGRE1DV1tcRF9WWlQIE

In [6]:
type(soup.find_all('table', attrs = {'class':'DataTable'})[0])

bs4.element.Tag

In [7]:
html_df = str(soup.find_all('table', attrs = {'class':'DataTable'})[0])

In [8]:
district02_df = pd.read_html(html_df)

In [9]:
district02_df = district02_df[0]

In [10]:
district02_df['Candidate'] = district02_df['Candidate'].str.replace("•","")
district02_df

,Candidate,Raised,Spent,Cash on Hand,Last Report
0,Tim Burchett (R) Incumbent Winner,"$1,336,276","$878,488","$593,678",12/31/2020
1,Renee Hoyos (D),"$812,784","$816,793",$210,12/31/2020


In [11]:
district02_df[['Candidate','party', 'incumbent', 'winner']] = district02_df.Candidate.str.extract(r'(\w+\s\w+\s)(\W\w\W\s?)(\s?\W?\w*)(\s?\W?\w*)' , expand = True) 

In [12]:
district02_df = district02_df.replace('','no')
district02_df = district02_df.replace(['(',')'], '')

In [13]:
district02_df

,Candidate,Raised,Spent,Cash on Hand,Last Report,party,incumbent,winner
0,Tim Burchett,"$1,336,276","$878,488","$593,678",12/31/2020,(R),Incumbent,Winner
1,Renee Hoyos,"$812,784","$816,793",$210,12/31/2020,(D),no,no


In [14]:
soup.find_all('li', {'class':'SubNav-item'})

[<li class="SubNav-item">
 <a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=TNS2&amp;spec=N">
                         Tennessee Senate <span aria-hidden="true">›</span>
 </a> </li>,
 <li class="SubNav-item">
 <a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=TN01&amp;spec=N">
                         Tennessee District 01 <span aria-hidden="true">›</span>
 </a> </li>,
 <li class="SubNav-item">
 <a class="SubNav-link SubNav-link--active" href="/races/summary?cycle=2020&amp;id=TN02&amp;spec=N">
                         Tennessee District 02 <span aria-hidden="true">›</span>
 </a> </li>,
 <li class="SubNav-item">
 <a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=TN03&amp;spec=N">
                         Tennessee District 03 <span aria-hidden="true">›</span>
 </a> </li>,
 <li class="SubNav-item">
 <a class="SubNav-link" href="/races/summary?cycle=2020&amp;id=TN04&amp;spec=N">
                         Tennessee District 04 <span aria-hidden="true">›<

In [15]:
#https://data36.com/scrape-multiple-web-pages-beautiful-soup-tutorial/
n = 1
all_districts_tn=pd.DataFrame()
for n in range(1,10):
    URL = f'https://www.opensecrets.org/races/summary?cycle=2020&id=TN0{n}&spec=N'
    res1 = requests.get(URL)
    soup1 = BS(res1.text)
    tn_districts = pd.read_html(str(soup1.find_all('table', attrs = {'class':'DataTable'})[0]))[0]
    tn_districts['districts'] = n
    tn_districts['state'] = 'TN'
    all_districts_tn = all_districts_tn.append(tn_districts)
    n+=1
tn_districts

,Candidate,Raised,Spent,Cash on Hand,Last Report,districts,state
0,Steve Cohen (D) • Incumbent • Winner,"$607,570","$457,311","$1,216,119",12/31/2020,9,TN
1,Charlotte Bergmann (R),"$99,147","$100,532","$2,566",12/31/2020,9,TN
2,Dennis Clark (I),"$9,991","$9,919",$40,11/23/2020,9,TN


In [16]:
all_districts_tn

,Candidate,Raised,Spent,Cash on Hand,Last Report,districts,state
0,Diana Harshbarger (R) • Winner,"$2,126,946","$1,869,100","$257,846",12/31/2020,1,TN
1,Blair Nicole Walsingham (D),"$140,209","$134,995","$5,215",10/14/2020,1,TN
0,Tim Burchett (R) • Incumbent • Winner,"$1,336,276","$878,488","$593,678",12/31/2020,2,TN
1,Renee Hoyos (D),"$812,784","$816,793",$210,12/31/2020,2,TN
0,Chuck Fleischmann (R) • Incumbent • Winner,"$1,051,653","$381,411","$1,880,341",12/31/2020,3,TN
1,Meg Gorman (D),"$85,843","$77,760","$8,083",12/31/2020,3,TN
0,Scott Desjarlais (R) • Incumbent • Winner,"$331,464","$392,499","$302,649",12/31/2020,4,TN
1,Christopher Hale (D),"$308,731","$302,996","$5,735",12/31/2020,4,TN
0,Jim Cooper (D) • Incumbent • Winner,"$936,569","$1,332,131","$272,934",12/31/2020,5,TN
0,John Rose (R) • Incumbent • Winner,"$1,050,429","$625,688","$454,375",12/31/2020,6,TN


In [17]:
all_districts_tn.Candidate.to_list()

['Diana Harshbarger (R)  • Winner',
 'Blair Nicole Walsingham (D)',
 'Tim Burchett (R)  • Incumbent • Winner',
 'Renee Hoyos (D)',
 'Chuck Fleischmann (R)  • Incumbent • Winner',
 'Meg Gorman (D)',
 'Scott Desjarlais (R)  • Incumbent • Winner',
 'Christopher Hale (D)',
 'Jim Cooper (D)  • Incumbent • Winner',
 'John Rose (R)  • Incumbent • Winner',
 'Mark Green (R)  • Incumbent • Winner',
 'Kiran Sreepada (D)',
 'Ronald Brown (I)',
 'Scott Vieira Jr (I)',
 'David Kustoff (R)  • Incumbent • Winner',
 'Erika Stotts Pearson (D)',
 'Steve Cohen (D)  • Incumbent • Winner',
 'Charlotte Bergmann (R)',
 'Dennis Clark (I)']

In [18]:
all_districts_tn[['name', 'party', 'nan', 'incumbent', 'winner']] = all_districts_tn.Candidate.str.extract(r'(\W?\w+?\s?\w+\s\w+\s)(\W\w\W\s?)(\s?\W?\w*)(\s?\W?\w*\s?\W?)?(\s?\w+)?' , expand = True) 

In [19]:
all_districts_tn['incumbent'].to_list()

[' Winner',
 '',
 ' Incumbent •',
 '',
 ' Incumbent •',
 '',
 ' Incumbent •',
 '',
 ' Incumbent •',
 ' Incumbent •',
 ' Incumbent •',
 '',
 '',
 '',
 ' Incumbent •',
 '',
 ' Incumbent •',
 '',
 '']

In [20]:
all_districts_tn = (all_districts_tn.drop(columns = ['nan'])
                    .fillna(False).
                    replace({' Winner': True , ' Incumbent •': True, '':False}))

In [21]:
all_districts_tn.head()

,Candidate,Raised,Spent,Cash on Hand,Last Report,districts,state,name,party,incumbent,winner
0,Diana Harshbarger (R) • Winner,"$2,126,946","$1,869,100","$257,846",12/31/2020,1,TN,Diana Harshbarger,(R),True,False
1,Blair Nicole Walsingham (D),"$140,209","$134,995","$5,215",10/14/2020,1,TN,Blair Nicole Walsingham,(D),False,False
0,Tim Burchett (R) • Incumbent • Winner,"$1,336,276","$878,488","$593,678",12/31/2020,2,TN,Tim Burchett,(R),True,True
1,Renee Hoyos (D),"$812,784","$816,793",$210,12/31/2020,2,TN,Renee Hoyos,(D),False,False
0,Chuck Fleischmann (R) • Incumbent • Winner,"$1,051,653","$381,411","$1,880,341",12/31/2020,3,TN,Chuck Fleischmann,(R),True,True


In [22]:
#getting all data from all states:
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
url2 = 'https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120'
r = requests.get(url2)
soup2 = BS(r.text)
r

<Response [200]>

In [23]:
num_of_dist = pd.read_html(str(soup2.find_all('table')))

In [24]:
num_of_dist = num_of_dist[0]
num_of_dist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   state            51 non-null     object
 1   representatives  51 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 944.0+ bytes


In [25]:
num_of_dist = num_of_dist.drop(index = 50)

In [26]:
num_of_dist['abbrev'] = states

In [27]:
num_of_dist

,state,representatives,abbrev
0,Alabama,7,AL
1,Alaska,1,AK
2,Arizona,9,AZ
3,Arkansas,4,AR
4,California,53,CA
5,Colorado,7,CO
6,Connecticut,5,CT
7,Delaware,1,DE
8,Florida,27,FL
9,Georgia,14,GA


In [28]:
representatives = num_of_dist['representatives'].to_list()
state_abbrev = num_of_dist['abbrev'].to_list()

In [29]:
num_of_dist['representatives'].sum()

435

In [30]:
reps_by_state = list(zip(state_abbrev,representatives))

In [31]:
reps_by_state

[('AL', 7),
 ('AK', 1),
 ('AZ', 9),
 ('AR', 4),
 ('CA', 53),
 ('CO', 7),
 ('CT', 5),
 ('DE', 1),
 ('FL', 27),
 ('GA', 14),
 ('HI', 2),
 ('ID', 2),
 ('IL', 18),
 ('IN', 9),
 ('IA', 4),
 ('KS', 4),
 ('KY', 6),
 ('LA', 6),
 ('ME', 2),
 ('MD', 8),
 ('MA', 9),
 ('MI', 14),
 ('MN', 8),
 ('MS', 4),
 ('MO', 8),
 ('MT', 1),
 ('NE', 3),
 ('NV', 4),
 ('NH', 2),
 ('NJ', 12),
 ('NM', 3),
 ('NY', 27),
 ('NC', 13),
 ('ND', 1),
 ('OH', 16),
 ('OK', 5),
 ('OR', 5),
 ('PA', 18),
 ('RI', 2),
 ('SC', 7),
 ('SD', 1),
 ('TN', 9),
 ('TX', 36),
 ('UT', 4),
 ('VT', 1),
 ('VA', 11),
 ('WA', 10),
 ('WV', 3),
 ('WI', 8),
 ('WY', 1)]

In [32]:
#i = 10
all_districts_us_10= pd.DataFrame()
all_districts_us_9 = pd.DataFrame()
for state in states:
    for i in range(1,54):
        sleep(0.1)
        if i < 10:
            try:
                URL2 = f'https://www.opensecrets.org/races/summary?cycle=2020&id={state}0{i}&spec=N'
                res2 = requests.get(URL2)
                soup3 = BS(res2.text)
                print(URL2,res2)
                us_districts1 = pd.read_html(str(soup3.find_all('table', attrs = {'class':'DataTable'})[0]))[0]
                us_districts1['state'] = state
                us_districts1['district'] = i
                all_districts_us_9 = all_districts_us_9.append(us_districts1)
            except IndexError:
                break
        if i >= 10:
            try:
                URL3 = f'https://www.opensecrets.org/races/summary?cycle=2020&id={state}{i}&spec=N'
                res3 = requests.get(URL3)
                soup4 = BS(res3.text)
                print(URL3, res3)
                us_districts2 = pd.read_html(str(soup4.find_all('table', attrs = {'class':'DataTable'})[0]))[0]
                us_districts2['state'] = state
                us_districts2['district'] = i
                all_districts_us_10 = all_districts_us_10.append(us_districts2)
            except IndexError:
                break
    i+=1


https://www.opensecrets.org/races/summary?cycle=2020&id=AL01&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=AL02&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=AL03&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=AL04&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=AL05&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=AL06&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=AL07&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=AL08&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=AK01&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=AK02&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=AZ01&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=AZ02&spec

https://www.opensecrets.org/races/summary?cycle=2020&id=FL04&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=FL05&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=FL06&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=FL07&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=FL08&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=FL09&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=FL10&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=FL11&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=FL12&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=FL13&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=FL14&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=FL15&spec

https://www.opensecrets.org/races/summary?cycle=2020&id=LA04&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=LA05&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=LA06&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=LA07&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=LA08&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=ME01&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=ME02&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=ME03&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=MD01&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=MD02&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=MD03&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=MD04&spec

https://www.opensecrets.org/races/summary?cycle=2020&id=NM02&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=NM03&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=NM04&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=NY01&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=NY02&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=NY03&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=NY04&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=NY05&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=NY06&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=NY07&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=NY08&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=NY09&spec

https://www.opensecrets.org/races/summary?cycle=2020&id=PA18&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=PA19&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=PA20&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=RI01&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=RI02&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=RI03&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=SC01&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=SC02&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=SC03&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=SC04&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=SC05&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=SC06&spec

https://www.opensecrets.org/races/summary?cycle=2020&id=WI01&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=WI02&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=WI03&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=WI04&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=WI05&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=WI06&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=WI07&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=WI08&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=WI09&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=WY01&spec=N <Response [200]>
https://www.opensecrets.org/races/summary?cycle=2020&id=WY02&spec=N <Response [200]>


In [116]:
frames = [all_districts_us_9, all_districts_us_10]
all_districts_us = pd.concat(frames)

In [117]:
all_districts_us

,Candidate,Raised,Spent,Cash on Hand,Last Report,state,district
0,Jerry Carl (R) • Winner,"$1,971,321","$1,859,349","$111,973",12/31/2020,AL,1
1,James Averhart (D),"$80,095","$78,973","$1,122",12/31/2020,AL,1
0,Barry Moore (R) • Winner,"$650,807","$669,368","-$13,633",12/31/2020,AL,2
1,Phyllis Harvey-Hall (D),"$56,050","$55,988",$0,12/31/2020,AL,2
0,Mike D Rogers (R) • Incumbent • Winner,"$1,193,111","$1,218,564","$502,234",12/31/2020,AL,3
...,...,...,...,...,...,...,...
0,Gerry Connolly (D) • Incumbent • Winner,"$1,998,592","$1,811,776","$2,891,921",12/31/2020,VA,11
1,Manga Anantatmula (R),"$282,152","$268,326","$13,826",12/31/2020,VA,11
2,Stevan Porter (L),$533,"$1,316",$149,12/31/2020,VA,11
0,Marilyn Strickland (D) • Winner,"$1,812,659","$1,780,633","$32,026",12/31/2020,WA,10


In [118]:
all_districts_us[['name', 'party', 'nan', 'incumbent', 'winner']] = all_districts_us.Candidate.str.extract(r'\W?(\w+\s?\w+?\s?\W?\w+\s)\W(\w)\W\s?\s?\W?(\w*)\s?\W?(\w*\s?\W?)?(\s?\w+)?' , expand = True)

In [119]:
all_districts_us

,Candidate,Raised,Spent,Cash on Hand,Last Report,state,district,name,party,nan,incumbent,winner
0,Jerry Carl (R) • Winner,"$1,971,321","$1,859,349","$111,973",12/31/2020,AL,1,Jerry Carl,R,,Winner,NaN
1,James Averhart (D),"$80,095","$78,973","$1,122",12/31/2020,AL,1,James Averhart,D,,,NaN
0,Barry Moore (R) • Winner,"$650,807","$669,368","-$13,633",12/31/2020,AL,2,Barry Moore,R,,Winner,NaN
1,Phyllis Harvey-Hall (D),"$56,050","$55,988",$0,12/31/2020,AL,2,Phyllis Harvey-Hall,D,,,NaN
0,Mike D Rogers (R) • Incumbent • Winner,"$1,193,111","$1,218,564","$502,234",12/31/2020,AL,3,Mike D Rogers,R,,Incumbent •,Winner
...,...,...,...,...,...,...,...,...,...,...,...,...
0,Gerry Connolly (D) • Incumbent • Winner,"$1,998,592","$1,811,776","$2,891,921",12/31/2020,VA,11,Gerry Connolly,D,,Incumbent •,Winner
1,Manga Anantatmula (R),"$282,152","$268,326","$13,826",12/31/2020,VA,11,Manga Anantatmula,R,,,NaN
2,Stevan Porter (L),$533,"$1,316",$149,12/31/2020,VA,11,Stevan Porter,L,,,NaN
0,Marilyn Strickland (D) • Winner,"$1,812,659","$1,780,633","$32,026",12/31/2020,WA,10,Marilyn Strickland,D,,Winner,NaN


In [120]:
all_districts_us = all_districts_us.reset_index()

In [121]:
all_districts_us

,index,Candidate,Raised,Spent,Cash on Hand,Last Report,state,district,name,party,nan,incumbent,winner
0,0,Jerry Carl (R) • Winner,"$1,971,321","$1,859,349","$111,973",12/31/2020,AL,1,Jerry Carl,R,,Winner,NaN
1,1,James Averhart (D),"$80,095","$78,973","$1,122",12/31/2020,AL,1,James Averhart,D,,,NaN
2,0,Barry Moore (R) • Winner,"$650,807","$669,368","-$13,633",12/31/2020,AL,2,Barry Moore,R,,Winner,NaN
3,1,Phyllis Harvey-Hall (D),"$56,050","$55,988",$0,12/31/2020,AL,2,Phyllis Harvey-Hall,D,,,NaN
4,0,Mike D Rogers (R) • Incumbent • Winner,"$1,193,111","$1,218,564","$502,234",12/31/2020,AL,3,Mike D Rogers,R,,Incumbent •,Winner
...,...,...,...,...,...,...,...,...,...,...,...,...,...
914,0,Gerry Connolly (D) • Incumbent • Winner,"$1,998,592","$1,811,776","$2,891,921",12/31/2020,VA,11,Gerry Connolly,D,,Incumbent •,Winner
915,1,Manga Anantatmula (R),"$282,152","$268,326","$13,826",12/31/2020,VA,11,Manga Anantatmula,R,,,NaN
916,2,Stevan Porter (L),$533,"$1,316",$149,12/31/2020,VA,11,Stevan Porter,L,,,NaN
917,0,Marilyn Strickland (D) • Winner,"$1,812,659","$1,780,633","$32,026",12/31/2020,WA,10,Marilyn Strickland,D,,Winner,NaN


In [122]:
all_districts_us['winner']= all_districts_us[all_districts_us['index'] == 0]['winner'].fillna(1)

In [123]:
all_districts_us['incumbent'] = all_districts_us['incumbent'].replace({'Winner': 0})

In [124]:
all_districts_us.winner.to_list()

[1,
 nan,
 1,
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 ' Winner',
 ' Winner',
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 ' Winner',
 nan,
 ' Winner',
 nan,
 1,
 nan,
 ' Winner',
 nan,
 ' Winner',
 ' Winner',
 nan,
 nan,
 1,
 nan,
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 nan,
 ' Winner',
 nan,
 nan,
 ' Winner',
 nan,
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 nan,
 nan,
 ' Winner',
 nan,
 nan,
 ' Winner',
 nan,
 1,
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 nan,
 ' Winner',
 nan,
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 nan,
 ' Winner',
 n

In [125]:
all_districts_us = (all_districts_us.drop(columns = ['nan'])
                    .fillna(False)
                    .replace({ ' Winner': 1 , 'Incumbent •': 1, '': 0}))

In [126]:
all_districts_us

,index,Candidate,Raised,Spent,Cash on Hand,Last Report,state,district,name,party,incumbent,winner
0,0,Jerry Carl (R) • Winner,"$1,971,321","$1,859,349","$111,973",12/31/2020,AL,1,Jerry Carl,R,0,1
1,1,James Averhart (D),"$80,095","$78,973","$1,122",12/31/2020,AL,1,James Averhart,D,0,False
2,0,Barry Moore (R) • Winner,"$650,807","$669,368","-$13,633",12/31/2020,AL,2,Barry Moore,R,0,1
3,1,Phyllis Harvey-Hall (D),"$56,050","$55,988",$0,12/31/2020,AL,2,Phyllis Harvey-Hall,D,0,False
4,0,Mike D Rogers (R) • Incumbent • Winner,"$1,193,111","$1,218,564","$502,234",12/31/2020,AL,3,Mike D Rogers,R,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
914,0,Gerry Connolly (D) • Incumbent • Winner,"$1,998,592","$1,811,776","$2,891,921",12/31/2020,VA,11,Gerry Connolly,D,1,1
915,1,Manga Anantatmula (R),"$282,152","$268,326","$13,826",12/31/2020,VA,11,Manga Anantatmula,R,0,False
916,2,Stevan Porter (L),$533,"$1,316",$149,12/31/2020,VA,11,Stevan Porter,L,0,False
917,0,Marilyn Strickland (D) • Winner,"$1,812,659","$1,780,633","$32,026",12/31/2020,WA,10,Marilyn Strickland,D,0,1


In [129]:
all_districts_us['incumbent'] = all_districts_us['incumbent'].astype(bool)
all_districts_us['winner'] = all_districts_us['winner'].astype(bool)
all_districts_us.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919 entries, 0 to 918
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         919 non-null    int64 
 1   Candidate     919 non-null    object
 2   Raised        919 non-null    object
 3   Spent         919 non-null    object
 4   Cash on Hand  919 non-null    object
 5   Last Report   919 non-null    object
 6   state         919 non-null    object
 7   district      919 non-null    int64 
 8   name          919 non-null    object
 9   party         919 non-null    object
 10  incumbent     919 non-null    bool  
 11  winner        919 non-null    bool  
dtypes: bool(2), int64(2), object(8)
memory usage: 73.7+ KB


In [130]:
x = 1
df = [0]
for x in range(1,50):
    df.append(all_districts_us[all_districts_us['state'] == states[x]]['winner'].astype(bool).sum())
len(df)

50

In [131]:
num_of_dist['winners'] = df
num_of_dist

,state,representatives,abbrev,winners
0,Alabama,7,AL,0
1,Alaska,1,AK,1
2,Arizona,9,AZ,9
3,Arkansas,4,AR,4
4,California,53,CA,53
5,Colorado,7,CO,7
6,Connecticut,5,CT,5
7,Delaware,1,DE,1
8,Florida,27,FL,27
9,Georgia,14,GA,14


In [132]:

#Finding and correcting districts with wrong amount of winners
num_of_dist[num_of_dist['winners']!=num_of_dist['representatives']]

,state,representatives,abbrev,winners
0,Alabama,7,AL,0
12,Illinois,18,IL,19
14,Iowa,4,IA,5
17,Louisiana,6,LA,7
20,Massachusetts,9,MA,10
21,Michigan,14,MI,15
24,Missouri,8,MO,9
29,New Jersey,12,NJ,13
31,New York,27,NY,29
34,Ohio,16,OH,18


In [133]:
ca_districts = all_districts_us[all_districts_us['state'] == 'MA']
ca_districts[ca_districts['winner']==True]

,index,Candidate,Raised,Spent,Cash on Hand,Last Report,state,district,name,party,incumbent,winner
245,0,Richard E Neal (D) • Incumbent • Winner,"$4,949,384","$5,873,122","$2,384,564",12/31/2020,MA,1,Richard E Neal,D,True,True
246,0,James P McGovern (D) • Incumbent • Winner,"$1,052,549","$1,089,412","$264,946",12/31/2020,MA,2,James P McGovern,D,True,True
248,0,Lori Trahan (D) • Incumbent • Winner,"$1,449,303","$1,232,437","$285,963",12/31/2020,MA,3,Lori Trahan,D,True,True
249,0,Jake Auchincloss (D) • Winner,"$2,608,686","$2,150,904","$457,782",12/31/2020,MA,4,Jake Auchincloss,D,False,True
251,0,Katherine Clark (D) • Incumbent • Winner,"$1,788,706","$1,514,877","$1,292,775",12/31/2020,MA,5,Katherine Clark,D,True,True
253,0,Seth Moulton (D) • Incumbent • Winner,"$1,289,805","$1,802,169","$245,668",12/31/2020,MA,6,Seth Moulton,D,True,True
255,0,Ayanna Pressley (D) • Incumbent • Winner,"$2,163,370","$1,748,432","$501,037",12/31/2020,MA,7,Ayanna Pressley,D,True,True
256,0,Stephen F Lynch (D) • Incumbent • Winner,"$905,614","$1,134,662","$1,213,384",12/31/2020,MA,8,Stephen F Lynch,D,True,True
257,0,Bill Keating (D) • Incumbent • Winner,"$708,680","$652,448","$1,504,778",12/31/2020,MA,9,Bill Keating,D,True,True
734,0,Bill Keating (D) • Winner (46.9% of vote),"$1,508,869","$1,505,516","$3,353",12/31/2010,MA,10,Bill Keating,D,True,True


In [134]:
#Drop IL district 19 index 730
#Drop IA district 5 index 187
#Drop LA district 7 index 224
#MA seems ok
#Drop MI dist 15 index 746
#Drop MO dist 9 index 315
#Drop NJ dist 13 index 753
#Drop NY dist 28 index 794
#Drop OH dist 17 index 821
#Drop PA dist 19 index 843
all_districts_us = all_districts_us.drop(index = [730,187,224,746,315,753,794,821,843])

In [135]:
#Now correct when including special elections
all_districts_us['winner'].astype(bool).sum()

440

In [136]:
all_districts_us.to_csv('all_districts_us')